In [1]:
import pandas as pd
import requests, warnings

In [2]:
def ag_get_data(query):
    warnings.simplefilter('ignore', requests.packages.urllib3.exceptions.InsecureRequestWarning)
    params = {"Database": "OIL", "Query": query}
    resp = requests.post('https://TST-QDEV-AP1.petroineos.local:5001/genericdata/Fetch/', json=params, verify=False)
    json_result = resp.json()
    df = pd.DataFrame(json_result)
    return df

In [3]:
adi_query = '''
select iir.COUNTRY, iir.UNIT_GROUP, sum(capacity) capacity from oil.refinery.RefineryUnitCapacityMonthly schedule
join oil.Refinery.RefineryUnit iir on iir.unit_id=schedule.unit_id
where DDate ='2022-02-01'
group by iir.COUNTRY, iir.UNIT_GROUP
order by iir.COUNTRY, iir.UNIT_GROUP
'''

In [42]:
iir_cdu_q = '''
select cast(PLANT_ID as int) manual, OWNER_NAME, PLANT_NAME, sum(U_CAPACITY) cdu from oil.Refinery.RefineryUnit
where UTYPE_DESC='Atmospheric Distillation'
group by PLANT_ID,OWNER_NAME, PLANT_NAME
'''

In [4]:
adi_capacities = ag_get_data(adi_query)

In [43]:
iir_cap = ag_get_data(iir_cdu_q)

In [11]:
h_comet_raw = pd.read_excel('refinery mapping/h_comet_capacities_raw.xlsx', header=6)

In [15]:
h_comet_cdu = h_comet_raw.groupby('Country')['ADU'].sum().reset_index()

In [17]:
h_comet_cdu.to_excel('cdu_comparison.xlsx')